In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats as st
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
%%time
#let's read the first training parquet
train_0 = pd.read_parquet('/kaggle/input/training-data/training/training0.parquet')
train_0.head()

In [ ]:
#because there are a lot of instances,I am going to keep only a fragment of these
train_0[train_0['session']==749999]
train_0.shape
train_0.session.unique().shape[0]

In [ ]:
#Let's see how many products have been seen in each session
train_0.groupby('session')['aid'].count().hist(bins = 10)

In [ ]:
%%time
#Now let's look how many unique products there are in each session
train_0.groupby('session')['aid'].nunique().hist()
plt.xlim([0,250])
#Most people look from 0 to 45 products usually 

In [ ]:
#convert integers 0,1,2 back to clicks,carts,orders for better visualization
types = ['clicks','carts','orders']
train_0['type'] = train_0.type.map(lambda i: types[i])
train_0

In [ ]:
#Now let's do the same for the testing data 
test_0 = pd.read_parquet('/kaggle/input/testing-data/testing/testing_final0.parquet')
test_0.head()

In [ ]:
#Same as before, Let's see how many products have been seen in each session
test_0.groupby('session')['aid'].count().hist(bins = 5)
plt.xlim([0,100])

In [ ]:
%%time
#Now let's look how many unique products there are in each session
test_0.groupby('session')['aid'].nunique().hist()
plt.xlim([0,60])
#the same 0-45 products 

Used two notebooks for inspiration, [this](https://www.kaggle.com/code/cdeotte/time-series-eda-users-and-real-sessions) from CHRIS DEOTTE 2nd place and [this](https://www.kaggle.com/code/radek1/eda-an-overview-of-the-full-dataset) from RADEK OSMULSKI.

In [ ]:
#Let's start!
#import libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

In [ ]:
train_0.shape

In [ ]:
train_0.info()

In [ ]:
train_0.describe()

In [ ]:
#check how many clicks , carts , orders have been made in total
train_0.type.value_counts()
#obviously there are a lot more clicks,less carts and even less orders

In [ ]:
#how many different products there are 
train_0.aid.value_counts().sort_values(ascending = False).head(20)

In [ ]:
%%time
#now let's check how many clicks,carts & orders per session
#There are a lot so let's create boxplots to check the distribution of their sums (#clicks,#carts,#orders)
#groupby session
#took a lot of time 
grouped = train_0.groupby('session')
grouped.describe().head(3)

In [ ]:
%%time
idtype = {'clicks':0,'carts':1,'orders':2}
train_0['type'] = train_0.type.map(lambda i: idtype[i])

train_0

In [ ]:
%%time
grouped2 = train_0.groupby('session')
grouped2.describe().head(3)

In [ ]:
#create boxplots
plt.figure(figsize=(10,10))
grouped2.count().head(5)

In [ ]:
%%time
group_type =train_0.groupby(['session', 'type']).size()
#much better with time 

In [ ]:
group_type 

In [ ]:
%%time
clicks = []
carts = []
orders = []
j=0
for i, el in enumerate(group_type.to_numpy()):
#     print(i)
#     print(1 in group_type[i].index)
    if(0 in group_type[i].index):
        clicks.append(group_type[i][0])
    else:
        clicks.append(0)
    if(1 in group_type[i].index):
        carts.append(group_type[i][1])
    else:
        carts.append(0)
    if(2 in group_type[i].index):
        orders.append(group_type[i][2])
    else:
        orders.append(0)
    #make it stop after 100.000 sessions to make it faster
    if(j==100000):
        break
    j+=1
###### added and the zeros because they might affect to see how many people actually make orders in relation to clicks etc.

In [ ]:
#after boxplots check correlation of number of clicks with number of orders or number of carts with number of orders
#find most popular products--->define popular (by clicks or by orders or by time watching it or by frequency of visits?)
print(len(clicks))
print(len(carts))
print(len(orders))
#now that we have all the data separately let's plot the distributions
plt.figure(figsize = (10,6))
plt.boxplot(clicks,vert = False)
plt.title('Clicks')


In [ ]:
def show_distribution(var_data,name):
  min_val = min(var_data)

  max_val = max(var_data)
  mean_val = np.mean(var_data)
  median_val = np.median(var_data)
  mode_val = st.mode(var_data) #most commonly used price
  print(name,':','Minimum:',min_val,'Maximum:',max_val,'Mean:',mean_val,'Median:',median_val,'Mode:',mode_val[0])
  fig,ax = plt.subplots(2,1,figsize = (10,6))
  ax[0].hist(var_data)
  ax[0].axvline(x=min_val,color='gray',linestyle = 'dashed',linewidth = 2)
  ax[0].axvline(x=max_val,color='gray',linestyle = 'dashed',linewidth = 2)
  ax[0].axvline(x=mean_val,color='red',linestyle = 'dashed',linewidth = 2)
  ax[0].axvline(x=median_val,color='yellow',linestyle = 'dashed',linewidth = 2)
  ax[0].axvline(x=mode_val[0],color='cyan',linestyle = 'dashed',linewidth = 2)
  ax[0].set_title(name)
  ax[0].set_ylabel('Frequency')
  ax[1].boxplot(var_data,vert = False)
  # ax[1].set_title(var_data.name)
  fig.show()

In [ ]:
show_distribution(clicks,'clicks')

In [ ]:
show_distribution(carts,'carts')

In [ ]:
show_distribution(orders,'orders')

In [ ]:
#same thing but let's remove 0 values to look at what we have 
print(clicks.count(0))
print(carts.count(0))
print(orders.count(0))
#A lot of times there weren't any carts or orders simply clicks

In [ ]:
#Let's remove these values and take a closer look 
clicks1 = [i for i in clicks if i != 0]
len(clicks1)

In [ ]:
carts1 = [i for i in carts if i != 0]
len(carts1)

In [ ]:
orders1 = [i for i in orders if i!=0]
len(orders1)

In [ ]:
show_distribution(clicks1,'clicks')

In [ ]:
show_distribution(carts1,'carts')

In [ ]:
show_distribution(orders1,'orders')

In [ ]:
#Because of outliers we can't see anything so lets remove the top 95% of observations
clicks2 = [i for i in clicks1 if i < 110]
len(clicks2)

In [ ]:
show_distribution(clicks2,'clicks')

In [ ]:
carts2 = [i for i in carts1 if i < 30]
len(carts2)

In [ ]:
show_distribution(carts2,'carts')

In [ ]:
orders2 = [i for i in orders1 if i < 10]
len(orders2)

In [ ]:
show_distribution(orders2,'orders')

### Timestamps

In [ ]:
train_0.ts.head(3)

In [ ]:
train_0.ts[0]

In [ ]:
import datetime
from datetime import datetime
#way of visualizing the dates with format yyyy/mm/dd -Example
datetime.fromtimestamp(train_0.ts[0]/1000)

In [ ]:
#Let's check when the training begins/ends.
print('Training begins:',datetime.fromtimestamp(train_0.ts.min()/1000))
print('Training ends:',datetime.fromtimestamp(train_0.ts.max()/1000))

In [ ]:
#same for testing
print('Testing begins:',datetime.fromtimestamp(test_0.ts.min()/1000))
print('Testing ends:',datetime.fromtimestamp(test_0.ts.max()/1000))

### Users and real sessions (from [this](https://www.kaggle.com/code/cdeotte/time-series-eda-users-and-real-sessions) notebook from kaggle)

In [ ]:
#need a gpu for these instances 
!nvidia-smi

In [ ]:
import matplotlib.patches as mpatches
import calendar

In [ ]:
#Let's take a random sample from the training data
#First get a list of the sessions
sessions = train_0['session'].unique()
sessions

In [ ]:
#get a sample of these sessions
import random
sample = random.sample(sessions.tolist(),len(sessions)//10)
sample[0:5]

In [ ]:
#for those random sessions, get their events from train_0
data_sample = train_0.loc[train_0.session.isin(sample)]
data_sample.head(5)

In [ ]:
data_sample.shape
#we have 3. something million data to continue

In [ ]:
#Like before get min-max dates
print(datetime.fromtimestamp(data_sample.ts.min()/1000))
print(datetime.fromtimestamp(data_sample.ts.max()/1000))

In [ ]:
%%time
#So for these dates we can create new columns instead of timestamp to get the day and hour (month,year hold no value for this analysis)
datetime_format = '%Y-%m-%d %H:%M:%S'
data_sample['date'] =data_sample['ts'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')).astype('datetime64[ns]')

# data_sample['hour'] = data_sample.date.datetime.hour
data_sample.head(5)

In [ ]:
data_sample['day'] = data_sample['date'].dt.day
data_sample.tail(5)

In [ ]:
data_sample['hour'] = data_sample.date.dt.hour
data_sample.head(5)

In [ ]:
data_sample['day_name'] = data_sample.date.dt.strftime('%A')

In [ ]:
data_sample.head(10)

In [ ]:
data_sample.info()

In [ ]:
%%time
#We want to display the transactions of the most active examples (top 20% ?)-->those may be outliers 
#Perhaps later check for those with transactions usually between normal numbers

grouped3 = data_sample.groupby('session').aid.count()
grouped3.head(10)
print('Max price is:',grouped3.max())
print('Min price is:',grouped3.min())
print('Mean price is:',grouped3.mean())
print('Mode price is:',grouped3.mode())
grouped3.values
#So in average,there are 40 transactions in every session.

In [ ]:
%%time
#Let's see which day we have the most events happening
grouped4 = data_sample.groupby(['session','day_name'])


In [ ]:
Days = grouped4.aid.count().to_dict()

In [ ]:
for i in Days.keys():
    print(Days[i])
    break

In [ ]:
%%time
Monday = pd.DataFrame({'Monday': (Days[i] for i in Days.keys() if i[1]=='Monday')})
Tuesday = pd.DataFrame({'Tuesday': (Days[i] for i in Days.keys() if i[1]=='Tuesday')})
Wednesday = pd.DataFrame({'Wednesday': (Days[i] for i in Days.keys() if i[1]=='Wednesday')})
Thursday = pd.DataFrame({'Thursday': (Days[i] for i in Days.keys() if i[1]=='Thursday')})
Friday = pd.DataFrame({'Friday': (Days[i] for i in Days.keys() if i[1]=='Friday')})
Saturday = pd.DataFrame({'Saturday': (Days[i] for i in Days.keys() if i[1]=='Saturday')})
Sunday = pd.DataFrame({'Sunday': (Days[i] for i in Days.keys() if i[1]=='Sunday')})


In [ ]:
Days_final  = pd.concat([Monday, Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday], axis=1)
Days_final

In [ ]:
Days_final.sum(axis = 0)


In [ ]:
fig = px.bar(Days_final.sum(axis = 0).sort_values(ascending = False), title="Distribution of events",width = 600,height = 400)
fig.show()

In [ ]:
grouped5 = data_sample.groupby(['session','type','day_name']).aid
grouped5.count()

In [ ]:
Days2 = grouped5.count().to_dict()

In [ ]:
#for clicks

Monday = pd.DataFrame({'Monday': (Days2[i] for i in Days2.keys() if ((i[2]=='Monday') & (i[1]==0)))})
Tuesday = pd.DataFrame({'Tuesday': (Days2[i] for i in Days2.keys() if((i[2]=='Tuesday') & (i[1]==0)))})
Wednesday = pd.DataFrame({'Wednesday': (Days2[i] for i in Days2.keys() if ((i[2]=='Wednesday') & (i[1]==0)))})
Thursday = pd.DataFrame({'Thursday': (Days2[i] for i in Days2.keys() if ((i[2]=='Thursday') & (i[1]==0)))})
Friday = pd.DataFrame({'Friday': (Days2[i] for i in Days2.keys() if ((i[2]=='Friday') & (i[1]==0)))})
Saturday = pd.DataFrame({'Saturday': (Days2[i] for i in Days2.keys() if ((i[2]=='Saturday') & (i[1]==0)))})
Sunday = pd.DataFrame({'Sunday': (Days2[i] for i in Days2.keys() if ((i[2]=='Sunday') & (i[1]==0)))})

In [ ]:
Days2_final  = pd.concat([Monday, Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday], axis=1)
Days2_final

In [ ]:
fig = px.bar(Days2_final.sum(axis = 0).sort_values(ascending = False), title="Distribution of clicks",width = 600,height = 400)
fig.show()

In [ ]:
Monday = pd.DataFrame({'Monday': (Days2[i] for i in Days2.keys() if ((i[2]=='Monday') & (i[1]==2)))})
Tuesday = pd.DataFrame({'Tuesday': (Days2[i] for i in Days2.keys() if((i[2]=='Tuesday') & (i[1]==2)))})
Wednesday = pd.DataFrame({'Wednesday': (Days2[i] for i in Days2.keys() if ((i[2]=='Wednesday') & (i[1]==2)))})
Thursday = pd.DataFrame({'Thursday': (Days2[i] for i in Days2.keys() if ((i[2]=='Thursday') & (i[1]==2)))})
Friday = pd.DataFrame({'Friday': (Days2[i] for i in Days2.keys() if ((i[2]=='Friday') & (i[1]==2)))})
Saturday = pd.DataFrame({'Saturday': (Days2[i] for i in Days2.keys() if ((i[2]=='Saturday') & (i[1]==2)))})
Sunday = pd.DataFrame({'Sunday': (Days2[i] for i in Days2.keys() if ((i[2]=='Sunday') & (i[1]==2)))})

In [ ]:
Days3_final  = pd.concat([Monday, Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday], axis=1)
Days3_final

In [ ]:
fig = px.bar(Days3_final.sum(axis = 0).sort_values(ascending = False), title="Distribution of orders",width = 600,height = 400)
fig.show()

In [ ]:
#for carts
Monday = pd.DataFrame({'Monday': (Days2[i] for i in Days2.keys() if ((i[2]=='Monday') & (i[1]==1)))})
Tuesday = pd.DataFrame({'Tuesday': (Days2[i] for i in Days2.keys() if((i[2]=='Tuesday') & (i[1]==1)))})
Wednesday = pd.DataFrame({'Wednesday': (Days2[i] for i in Days2.keys() if ((i[2]=='Wednesday') & (i[1]==1)))})
Thursday = pd.DataFrame({'Thursday': (Days2[i] for i in Days2.keys() if ((i[2]=='Thursday') & (i[1]==1)))})
Friday = pd.DataFrame({'Friday': (Days2[i] for i in Days2.keys() if ((i[2]=='Friday') & (i[1]==1)))})
Saturday = pd.DataFrame({'Saturday': (Days2[i] for i in Days2.keys() if ((i[2]=='Saturday') & (i[1]==1)))})
Sunday = pd.DataFrame({'Sunday': (Days2[i] for i in Days2.keys() if ((i[2]=='Sunday') & (i[1]==1)))})

In [ ]:
Days4_final  = pd.concat([Monday, Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday], axis=1)
Days4_final

In [ ]:
fig = px.bar(Days4_final.sum(axis = 0).sort_values(ascending = False), title="Distribution of carts",width = 600,height = 400)
fig.show()

In [ ]:
#Let's organize them per hour 
grouped6 = data_sample.groupby(['session','hour']).aid
grouped6.count()

In [ ]:
Hours = grouped6.count().to_dict()


In [ ]:
df1 = pd.DataFrame({'hour 0': (Hours[i] for i in Hours.keys() if i[1]==0)})
for k in range(1,24):
    df = pd.DataFrame({'hour '+str(k): (Hours[i] for i in Hours.keys() if i[1]==k)})
    df1 = pd.concat([df1,df], axis=1)

In [ ]:
df1.sum()

In [ ]:
plt.figure(figsize = (8,4))
df1.sum().sort_values(ascending = False).plot(kind = 'bar')
plt.title('Events')
plt.ylabel('Number of events')
plt.show()
#we see that we have more events at noon or at lunch (perhaos break time ?)

In [ ]:
grouped7 = data_sample.groupby(['session','type','hour']).aid
grouped7.count()

In [ ]:
Hours0 = grouped7.count().to_dict()


In [ ]:
%%time
clicks = []
carts = []
orders = []
for k in range(0,24):
#     print('mpika')
    #data for clicks only
    temp1 = sum(list(Hours0[i] for i in Hours0.keys() if ((i[2]==k) & (i[1]==0))))
    clicks.append(temp1)
    temp2 = sum(list(Hours0[i] for i in Hours0.keys() if ((i[2]==k) & (i[1]==1))))
    carts.append(temp2)
    temp3 = sum(list(Hours0[i] for i in Hours0.keys() if ((i[2]==k) & (i[1]==2))))
    orders.append(temp3)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
fig = make_subplots(rows = 1,cols = 3,subplot_titles=("Clicks", "Carts", "Orders"))
fig.add_trace(
        go.Bar(x=np.arange(0,24), y=clicks),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=np.arange(0,24), y=carts),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=np.arange(0,24), y=orders),
    row=1, col=3
)